In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# Carregando arquivos Excel
excel_file1 = r'C:\Users\pedro\OneDrive\Desktop\Gráficos\Agua_bb_adulto.xlsx'
excel_file2 = r'C:\Users\pedro\OneDrive\Desktop\Gráficos\Bosque_bb_adulto.xlsx'

# Listas de abas para cada tabela
abas_agua = ['Gravador 01', 'Gravador 04', 'Gravador 05', 'Gravador 06']
abas_bosque = ['Gravador 01', 'Gravador 03', 'Gravador 04', 'Gravador 06', 'Gravador 07', 'Gravador 09']

# Carregando DataFrames de cada aba da tabela Água
df_agua = []
for aba in abas_agua:
    df_temp = pd.read_excel(excel_file1, sheet_name=aba)
    df_temp['Tabela'] = 'Água'
    df_agua.append(df_temp)

# Carregando DataFrames de cada aba da tabela Bosque
df_bosque = []
for aba in abas_bosque:
    df_temp = pd.read_excel(excel_file2, sheet_name=aba)
    df_temp['Tabela'] = 'Bosque'
    df_bosque.append(df_temp)

# Concatenando os DataFrames
combined_df_agua = pd.concat(df_agua, ignore_index=True)
combined_df_bosque = pd.concat(df_bosque, ignore_index=True)
combined_df = pd.concat([combined_df_agua, combined_df_bosque], ignore_index=True)

# Convertendo a coluna de Data para formato de data e Hora para datetime
combined_df['Data'] = pd.to_datetime(combined_df['Data'], format='%d/%m/%y')
combined_df['Hora'] = pd.to_datetime(combined_df['Hora'], format='%H:%M:%S')

# # Filtrando dados por vocalização e faixa horária
# horario_manha = (4, 6)
# horario_tarde = (14, 17)

# Filtrando para vocalizações 'phee adulto' das 4:00-6:00 e 14:00-17:00
filtered_data_phee_manha = combined_df[
    (combined_df['Classificacao'] == 'phee adulto') & 
    (combined_df['Hora'].dt.hour >= horario_manha[0]) & (combined_df['Hora'].dt.hour < horario_manha[1])
]

filtered_data_phee_tarde = combined_df[
    (combined_df['Classificacao'] == 'phee adulto') & 
    (combined_df['Hora'].dt.hour >= horario_tarde[0]) & (combined_df['Hora'].dt.hour < horario_tarde[1])
]
# Definindo as faixas horárias
horario_manha = (4, 6)  # das 4:00 às 6:00
horario_tarde = (14, 17)  # das 14:00 às 17:00

# Criando uma figura com dois subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Garantindo que as faixas horárias sejam tratadas como uma variável categórica com ordem definida
faixas_bosque_order = ['04:00-06:00', '14:00-17:00']

# Filtra o DataFrame para o grupo 'Bosque' e seleciona as faixas de horário desejadas
bosque_data = combined_df[(combined_df['Tabela'] == 'Bosque') & (combined_df['Classificacao'] == 'phee adulto') & 
                          ((combined_df['Hora'].dt.hour >= horario_manha[0]) & (combined_df['Hora'].dt.hour < horario_manha[1]) | 
                           (combined_df['Hora'].dt.hour >= horario_tarde[0]) & (combined_df['Hora'].dt.hour < horario_tarde[1]))]

# Adiciona uma coluna para indicar a faixa de horário
bosque_data['Faixa Horária'] = bosque_data['Hora'].apply(lambda x: '04:00-06:00' if horario_manha[0] <= x.hour < horario_manha[1] else '14:00-17:00')

# Convertendo a coluna 'Faixa Horária' para uma variável categórica com a ordem definida
bosque_data['Faixa Horária'] = pd.Categorical(bosque_data['Faixa Horária'], categories=faixas_bosque_order, ordered=True)

# Cria o boxplot para o grupo 'Bosque' com cor verde
sns.boxplot(data=bosque_data, x='Faixa Horária', y='Tempototal', ax=axes[0], showfliers=False, order=faixas_bosque_order, palette={'04:00-06:00': 'green', '14:00-17:00': 'green'})
axes[0].set_title("Bosque - Comparação de Duração por Horário")
axes[0].set_xlabel("Faixa Horária")
axes[0].set_ylabel("Duração (m)")

# Atualizando os rótulos do eixo X
axes[0].set_xticklabels(['Manhã', 'Tarde'])

# Teste t de Student para a comparação das faixas de horário (4:00-6:00 vs 14:00-17:00) no grupo Bosque
t_statistic_phee_bosque_horarios, p_value_phee_bosque_horarios = ttest_ind(
    bosque_data[bosque_data['Faixa Horária'] == '04:00-06:00']['Tempototal'],
    bosque_data[bosque_data['Faixa Horária'] == '14:00-17:00']['Tempototal']
)

# Filtra o DataFrame para o grupo 'Água' e seleciona as faixas de horário desejadas
agua_data = combined_df[(combined_df['Tabela'] == 'Água') & (combined_df['Classificacao'] == 'phee adulto') & 
                        ((combined_df['Hora'].dt.hour >= horario_manha[0]) & (combined_df['Hora'].dt.hour < horario_manha[1]) | 
                         (combined_df['Hora'].dt.hour >= horario_tarde[0]) & (combined_df['Hora'].dt.hour < horario_tarde[1]))]

# Adiciona uma coluna para indicar a faixa de horário
agua_data['Faixa Horária'] = agua_data['Hora'].apply(lambda x: '04:00-06:00' if horario_manha[0] <= x.hour < horario_manha[1] else '14:00-17:00')

# Convertendo a coluna 'Faixa Horária' para uma variável categórica com a ordem definida
agua_data['Faixa Horária'] = pd.Categorical(agua_data['Faixa Horária'], categories=faixas_bosque_order, ordered=True)

# Cria o boxplot para o grupo 'Água' com cor azul
sns.boxplot(data=agua_data, x='Faixa Horária', y='Tempototal', ax=axes[1], showfliers=False, order=faixas_bosque_order, palette={'04:00-06:00': 'blue', '14:00-17:00': 'blue'})
axes[1].set_title("Água - Comparação de Duração por Horário")
axes[1].set_xlabel("Faixa Horária ")
axes[1].set_ylabel("Duração (m)")

# Atualizando os rótulos do eixo X
axes[1].set_xticklabels(['Manhã', 'Tarde'])

# Teste t para a comparação de horários no grupo Água (04:00-06:00 vs 14:00-17:00)
t_statistic_phee_agua_horarios, p_value_phee_agua_horarios = ttest_ind(
    agua_data[agua_data['Faixa Horária'] == '04:00-06:00']['Tempototal'],
    agua_data[agua_data['Faixa Horária'] == '14:00-17:00']['Tempototal']
)

# Anotando os resultados dos testes t nos gráficos
axes[0].annotate(f'Test t: {t_statistic_phee_bosque_horarios:.2f}\np-value: {p_value_phee_bosque_horarios:.3f}', 
                 xy=(0.5, 0.9), xycoords='axes fraction', ha='center', va='center', weight='bold', fontsize=13)

axes[1].annotate(f'Test t: {t_statistic_phee_agua_horarios:.2f}\np-value: {p_value_phee_agua_horarios:.3f}', 
                 xy=(0.5, 0.9), xycoords='axes fraction', ha='center', va='center', weight='bold', fontsize=13)

# Ajustando o layout
plt.tight_layout()

# Exibindo os subplots
plt.show()